In [ ]:
import yfinance as yf
df = yf.download("EURGBP=X", start="2023-04-01", end="2023-07-09", interval="1d")

In [ ]:
# Relative strength index manually

def rsi(df, periods = 14, ema = True):
    close_delta = df['Close'].diff()
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    if ema == True:
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return rsi

In [ ]:
# Commodity channel index

In [ ]:
# Average true range

In [ ]:
# Using pandas ta lib

import pandas_ta as ta

df.ta.rsi(length=14, append=True)
df.ta.cci(length=20, append=True)
df.ta.atr(length=14, append=True)